In [1]:
from tqdm import tqdm
from time import sleep
import requests
import json
import pandas as pd
import numpy as np

In [2]:
header = {
    'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.134 YaBrowser/22.7.0.1842 Yowser/2.5 Safari/537.36',
    'content-type': 'application/json',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ru,en;q=0.9'
}

In [3]:
def log_error(status_code):
        print('ERROR: ' + (requests.status_codes._codes[status_code])[0], file=sys.stderr)

In [4]:
def get_page_content(url, n_attempts=5, t_sleep=1, header=None):
    for _ in range(n_attempts):
        r = requests.get(url, headers=header)
        if r.ok and r:
            return r
        
        log_error(r.status_code)
        sleep(t_sleep)

    return None

In [5]:
def get_price(page):
    # Тут я проверяю, есть ли товар в наличии
    if page['available']['online']['warehouse_codes'] or page['available']['offline']['stores']:
        if page['old_price'] is None:
            return (int(page['price']['price']), None)
        else:
            return (int(page['old_price']['price']), int(page['price']['price']))
    else:
        return (None, None)

In [6]:
def parse_lego(city):
    #MOW - Москва и Московская область
    #SPE - Санкт-Петербург и Ленинградская область
    product_id = []
    names = []
    prices = []
    promo_prices = []
    links = []
    for i in tqdm(range(0, 1950, 100)):
        url = f'https://api.detmir.ru/v2/products?filter=categories[].alias:lego;promo:false;withregion:RU-{city}' + \
                f'&expand=meta.facet.ages.adults,meta.facet.gender.adults,webp&meta=*&limit=100&offset={i}&sort=popularity:desc'
        content = get_page_content(url, n_attempts=3, t_sleep=1, header=header)

        for el in content.json()['items']:
            product_id.append(int(el['productId']))

            names.append(str(el['title']))

            price = get_price(el)
            prices.append(price[0])
            promo_prices.append(price[1])

            links.append(str(el['link']['web_url']))

    return {'id': product_id, 'title': names, 'price': prices, 'promo_price': promo_prices, 'url': links}

In [7]:
msk_data = parse_lego('MOW')
msk_data['city'] = ['Moscow'] * len(msk_data['price'])
spb_data = parse_lego('SPE')
spb_data['city'] = ['Saint-Petersburg'] * len(msk_data['price'])

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.51it/s]


In [8]:
df_msk = pd.DataFrame(data=msk_data)
df_spb = pd.DataFrame(data=spb_data)

In [9]:
df_lego = pd.concat([df_msk, df_spb], ignore_index=True)

In [10]:
df_lego.to_csv('lego.csv', encoding='utf-8-sig', index=False) 

In [11]:
df_lego

,id,title,price,promo_price,url,city
0,3092803,Конструктор LEGO DUPLO Town Грузовой поезд (10...,13789.0,12399.0,https://www.detmir.ru/product/index/id/3092803/,Moscow
1,3187781,Конструктор LEGO Minecraft Шахта крипера 21155,8989.0,NaN,https://www.detmir.ru/product/index/id/3187781/,Moscow
2,3405527,Конструктор LEGO DUPLO My First Поезд для живо...,1909.0,1329.0,https://www.detmir.ru/product/index/id/3405527/,Moscow
3,3244569,Конструктор LEGO Super Mario Приключения вмест...,5989.0,3979.0,https://www.detmir.ru/product/index/id/3244569/,Moscow
4,3596786,Конструктор LEGO Friends Волшебная акробатика ...,2029.0,1299.0,https://www.detmir.ru/product/index/id/3596786/,Moscow
...,...,...,...,...,...,...
3855,619141,Конструктор LEGO Juniors Ралли на гоночных авт...,NaN,NaN,https://www.detmir.ru/product/index/id/619141/,Saint-Petersburg
3856,619181,Конструктор LEGO Juniors Стройка (10667),NaN,NaN,https://www.detmir.ru/product/index/id/619181/,Saint-Petersburg
3857,619201,Конструктор LEGO DUPLO Town Лесной заповедник ...,NaN,NaN,https://www.detmir.ru/product/index/id/619201/,Saint-Petersburg
3858,619231,Конструктор LEGO DUPLO Town Уточки в лесу (10581),NaN,NaN,https://www.detmir.ru/product/index/id/619231/,Saint-Petersburg
